# Import libraries

In [1]:
import os
import torch
from torchvision import transforms
from src.data_loader.freihand_loader import F_DB
from src.models.baseline_model import BaselineModel
from torch.utils.data import DataLoader
from src.data_loader.utils import convert_2_5D_to_3D, get_root_depth, convert_to_2_5D
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CometLogger
from src.visualization.visualize import plot_hand
from src.constants import MASTER_THESIS_DIR, FREIHAND_DATA
from src.utils import read_json
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display

# Dataset

In [2]:
f_db = F_DB(
    root_dir=os.path.join(FREIHAND_DATA, "training", "rgb"),
    labels_path=os.path.join(FREIHAND_DATA, "training_xyz.json"),
    camera_param_path=os.path.join(FREIHAND_DATA, "training_K.json"),
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            #             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    ),
)

# Label Visualization

In [3]:
@interact(id=widgets.IntSlider(min=0, max=len(f_db), step=1, value=0))
def visualize_sample(id):
    A = f_db[id]["joints"]
    s = f_db[id]["scale"]
    K = f_db[id]["K"]
    img = transforms.ToPILImage('RGB')(f_db[id]["image"])
    display(img)
    Axy = f_db[id]["joints_3D"]
    fig = plt.figure(figsize=(10,10))
    gs = fig.add_gridspec(5, 5)
    ax1 = fig.add_subplot(gs[1:3,:2])
    ax1.set_title('2d pose')
    plot_hand(ax1, np.array(A))
    ax2 = fig.add_subplot(gs[:,2:], projection='3d')
    ax2.set_title('3D pose')
    plot_hand(ax2, np.array(Axy),plot_3d=True)
    plt.show()

interactive(children=(IntSlider(value=0, description='id', max=32560), Output()), _dom_classes=('widget-intera…

In [18]:
id = 13053
joints25D = f_db[id]["joints"]
scale = f_db[id]["scale"]
K = f_db[id]["K"]
true_joints_3D = f_db[id]["joints_3D"]

In [29]:
from typing import NewType, Tuple
PARENT_JOINT = 0
CHILD_JOINT = 2
JOINTS_25D = NewType(
    "Joints_25D", torch.tensor
)  # shape (points, 3) , third dimension set to unity.
SCALE = NewType("scale", torch.tensor)  # shape (1)
JOINTS_3D = NewType("Joints_3D", torch.tensor)  # shape  (pints, 3)
def get_root_depth(joints_25D: JOINTS_25D) -> torch.Tensor:
    x_n, y_n, Z_n = joints_25D[PARENT_JOINT, :]
    x_m, y_m, Z_m = joints_25D[CHILD_JOINT, :]
    print("x_n={}, y_n={}, Z_n={}".format(x_n, y_n, Z_n))
    print("x_m={}, y_m={}, Z_m={}".format(x_m, y_m, Z_m))
    C = 1
    a = (x_n - x_m) ** 2 + (y_n - y_m) ** 2
    b = Z_n * (x_n ** 2 + y_n ** 2 - x_n * x_m - y_n * y_m) + Z_m * (
        x_m ** 2 + y_m ** 2 - x_n * x_m - y_n * y_m
    )
    c = (
        (x_n * Z_n - x_m * Z_m) ** 2
        + (y_n * Z_n - y_m * Z_m) ** 2
        + (Z_n - Z_m) ** 2
        - C
    )
    print("a={},b={},c={}".format(a,b,c))
    Z_root = 0.5 * (-b + (b ** 2 - 4 * a * c) ** 0.5) / a
    # print(a,b,c)
    # print(Z_root)
    return Z_root

In [30]:
get_root_depth(joints25D)

x_n=105.68982696533203, y_n=145.85731506347656, Z_n=0.0
x_m=133.1248779296875, y_m=115.64970397949219, Z_m=-0.3389134705066681
a=1665.1817626953125,b=-53.814430236816406,c=3570.9990234375


tensor(nan)

In [27]:
def convert_to_2_5D(
    camera_param: torch.tensor, joints_3D: torch.tensor
) -> Tuple[JOINTS_25D, SCALE]:
    # Using the distance between and second coordinate as scale. (i.e. length of index fingure.)
    scale = (((joints_3D[CHILD_JOINT] - joints_3D[PARENT_JOINT]) ** 2).sum()) ** 0.5
    print(scale)
    # Using the equation (1/Z)*K@(J_3D) = J_2D
    # Below Joints3D is the tensor of 21 x 3, where as J_3D is 3 x 1.
    # joints3D.T -> 3 x 21
    # camera param and K -> 3x3
    #  camera_param@(joints3D.T).T -> 21 x 3
    # joints3D[:,-1:] -> 21 x 1
    # joints25D -> 21 x3 with the third dimesion as 1.
    joints_25D = ((camera_param @ (joints_3D.T)).T) / joints_3D[:, -1:]
    # adding the .5 th dimension relative to the root, here it is the 0th coordinate.
    # scale normalization of the relative z component. As the xy coordinate are not affected by scale.
    new_z = joints_25D[:,-1]
    print(new_z)
    joints_25D[:, -1] = (joints_3D[:, -1] - joints_3D[0, -1]) / scale
    return joints_25D, scale

In [28]:
convert_to_2_5D(K, true_joints_3D)

tensor(0.0598)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.])


(tensor([[105.6898, 145.8573,   0.0000],
         [123.9480, 130.8463,  -0.1886],
         [133.1249, 115.6497,  -0.3389],
         [134.2723, 102.5088,  -0.5983],
         [144.1983,  84.7234,  -0.9177],
         [120.6423,  90.8710,  -0.3325],
         [127.6807,  76.2137,  -0.5652],
         [134.8043,  75.1681,  -0.8107],
         [144.2101,  80.5108,  -1.1171],
         [106.9892,  86.3753,  -0.5023],
         [112.0367,  73.6854,  -0.7649],
         [119.7553,  77.4094,  -0.9551],
         [131.0435,  88.9040,  -1.1144],
         [ 92.2837,  93.9233,  -0.6160],
         [ 96.6239,  80.0022,  -0.8019],
         [104.4161,  82.4741,  -1.0023],
         [115.3614,  94.2690,  -1.1157],
         [ 83.0288, 103.1064,  -0.7117],
         [ 82.5331,  93.2741,  -0.8486],
         [ 88.0130,  91.8171,  -0.9808],
         [ 99.4384,  95.6932,  -1.0914]]),
 tensor(0.0598))

In [17]:
convert_2_5D_to_3D(joints25D,scale,K)

tensor([[105.6898, 145.8573,   0.0000],
        [123.9480, 130.8463,  -0.1886],
        [133.1249, 115.6497,  -0.3389],
        [134.2723, 102.5088,  -0.5983],
        [144.1983,  84.7234,  -0.9177],
        [120.6423,  90.8710,  -0.3325],
        [127.6807,  76.2137,  -0.5652],
        [134.8043,  75.1681,  -0.8107],
        [144.2101,  80.5108,  -1.1171],
        [106.9892,  86.3753,  -0.5023],
        [112.0367,  73.6854,  -0.7649],
        [119.7553,  77.4094,  -0.9551],
        [131.0435,  88.9040,  -1.1144],
        [ 92.2837,  93.9233,  -0.6160],
        [ 96.6239,  80.0022,  -0.8019],
        [104.4161,  82.4741,  -1.0023],
        [115.3614,  94.2690,  -1.1157],
        [ 83.0288, 103.1064,  -0.7117],
        [ 82.5331,  93.2741,  -0.8486],
        [ 88.0130,  91.8171,  -0.9808],
        [ 99.4384,  95.6932,  -1.0914]])

In [24]:
true_joints_3D[0,-1]

tensor(0.7288)

tensor([ 0.0000, -0.1887, -0.3391, -0.5986, -0.9182, -0.3327, -0.5655, -0.8111,
        -1.1177, -0.5026, -0.7654, -0.9556, -1.1150, -0.6164, -0.8023, -1.0028,
        -1.1163, -0.7121, -0.8491, -0.9813, -1.0920])